In [2]:
import os

In [3]:
%pwd

'c:\\Users\\devan\\Downloads\\Brain-Tumor-\\research'

In [25]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\devan\\Downloads\\Brain-Tumor-'

In [92]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict
import os

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float       # added
    params_augmentation: Dict         # added (dict from params.yaml)


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


from Brain_tumor_classifier.constants import *
from Brain_tumor_classifier.utils.common import read_yaml, create_directories
import tensorflow as tf


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config



    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        # Use the train folder path you defined in config.yaml
        training_data = Path(self.config.data_transformation.train_data_dir)

        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=training_data,
            params_epochs=int(params.EPOCHS),
            params_batch_size=int(params.BATCH_SIZE),
            params_is_augmentation=bool(params.AUGMENTATION),
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=float(params.LEARNING_RATE),
            params_augmentation=params.AUGMENTATION
        )

        return training_config




import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]




import urllib.request as request
from zipfile import ZipFile
import time






class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        # load the prepared updated base model (no compile assumptions)
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False
        )

        # compile with run_eagerly=True for debugging (temporary)
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"],
            run_eagerly=True   # <--- set this
        )

        
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=tuple(self.config.params_image_size[:2]),
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=str(self.config.training_data),
            subset="validation",
            shuffle=False,
            class_mode="categorical",
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs,
                **(self.config.params_augmentation or {})
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=str(self.config.training_data),
            subset="training",
            shuffle=True,
            class_mode="categorical",
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )




try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e


[2025-08-28 23:37:33,058: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-28 23:37:33,066: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-28 23:37:33,066: INFO: common: created directory at: artifacts]
[2025-08-28 23:37:33,066: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-08-28 23:37:33,066: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-08-28 23:37:33,066: INFO: common: created directory at: artifacts\training]
Found 39 images belonging to 2 classes.
Found 163 images belonging to 2 classes.
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.6104 - loss: 1.2430[2025-08-28 23:38:07,143: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.sav

c:\Users\devan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


[2025-08-28 23:38:16,724: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 658ms/step - accuracy: 0.4062 - loss: 1.2328 - val_accuracy: 0.7812 - val_loss: 0.5472
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.5944 - loss: 0.8824[2025-08-28 23:38:47,998: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
5/5 ━━━━━━━━━━━━━━━━━━━━ 31s 8s/step - accuracy: 0.6183 - loss: 0.8364 - val_accuracy: 0.7188 - val_loss: 0.5075
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━